In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [3]:
file_path=r"D:\SEMESTER 3\PYTHON G\Assignment 22\processes2.csv"
df = pd.read_csv(file_path)


print(df.head())

   Unnamed: 0     name  year  selling_price  km_driven    fuel seller_type  \
0           0   Maruti  2014         450000     145500  Diesel  Individual   
1           2  Hyundai  2010         225000     127000  Diesel  Individual   
2           4  Hyundai  2017         440000      45000  Petrol  Individual   
3           7   Toyota  2011         350000      90000  Diesel  Individual   
4           8     Ford  2013         200000     169000  Diesel  Individual   

  transmission        owner  seats  max_power (in bph) Mileage Unit  Mileage  \
0       Manual  First Owner      5               74.00         kmpl    23.40   
1       Manual  First Owner      5               90.00         kmpl    23.00   
2       Manual  First Owner      5               81.86         kmpl    20.14   
3       Manual  First Owner      5               67.10         kmpl    23.59   
4       Manual  First Owner      5               68.10         kmpl    20.00   

   Engine (CC)  
0         1248  
1         1396  

In [ ]:
#dropping unnecessary columns
df.drop(columns=['Unnamed: 0', 'name'], inplace=True, errors='ignore')



In [5]:
#checking null values and duplicates
print(df.isnull().sum())
print("Number of duplicate rows:", df.duplicated().sum())


Unnamed: 0            0
name                  0
year                  0
selling_price         0
km_driven             0
fuel                  0
seller_type           0
transmission          0
owner                 0
seats                 0
max_power (in bph)    0
Mileage Unit          0
Mileage               0
Engine (CC)           0
dtype: int64
Number of duplicate rows: 0


In [ ]:
#taking only relevant columns
df = df[['year', 'km_driven', 'Engine (CC)', 'max_power (in bph)', 'selling_price', 'fuel', 'transmission']]
df.columns = ['year', 'mileage', 'engineSize', 'maxPower', 'price', 'fuel', 'transmission']  


In [8]:
#doing OneHotCosing to convert categorocal values to numerical values
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first', sparse_output=False)

encoded_features = ohe.fit_transform(df[['fuel', 'transmission']])

encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(['fuel', 'transmission']))

df = df.drop(columns=['fuel', 'transmission'])
df = pd.concat([df, encoded_df], axis=1)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [11]:
#predicting values
y_pred = model.predict(X_test)


In [13]:
#calculating prefomance matrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared Score (R²):", r2)


Mean Absolute Error (MAE): 85086.66254726479
Mean Squared Error (MSE): 12469038820.940706
R-squared Score (R²): 0.736216547821173


In [15]:
import pickle

# Save the trained model to a file
with open("model_lr.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved successfully as 'model_lr.pkl'!")


Model saved successfully as 'model_lr.pkl'!


In [18]:
#making model more accuarate by removing outliers
# Function to detect and remove outliers using IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile (25%)
    Q3 = df[column].quantile(0.75)  # Third quartile (75%)
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtering data within the IQR range
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    print(f"✅ Removed {df.shape[0] - df_cleaned.shape[0]} outliers from '{column}'")
    return df_cleaned

# Removing outliers from key numerical columns
df = remove_outliers(df, 'price')
df = remove_outliers(df, 'mileage')
df = remove_outliers(df, 'engineSize')
df = remove_outliers(df, 'maxPower')


✅ Removed 0 outliers from 'price'
✅ Removed 0 outliers from 'mileage'
✅ Removed 0 outliers from 'engineSize'
✅ Removed 0 outliers from 'maxPower'


In [20]:

X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [24]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📌 Updated Mean Absolute Error (MAE):", mae)
print("📌 Updated Mean Squared Error (MSE):", mse)
print("📌 Updated R-squared Score (R²):", r2)


📌 Updated Mean Absolute Error (MAE): 78273.73561301849
📌 Updated Mean Squared Error (MSE): 9841399767.623354
📌 Updated R-squared Score (R²): 0.7700723664590934


In [25]:
#converting this file to pickle file
import pickle

# Save the newly trained model to a file
with open("model_lr.pkl", "wb") as file:
    pickle.dump(model, file)

print("Updated Model saved successfully as 'model_lr.pkl'!")


Updated Model saved successfully as 'model_lr.pkl'!
